## ODE battery model

A simple ODE battery model. Reservoir model used, which represents a battery as two reservoirs of charge (one for anode, one for cathode). This model is described using:

$$ \frac{dx_n}{dt} = -\frac{I(t)}{Q_n} $$
$$ \frac{dx_p}{dt} = -\frac{I(t)}{Q_p} $$
where:
$x_n$ and $x_p$ are dimensionless, stoichiometries of the negative and positive electrodes, where $x_n(0) = x_n0$ (same for $x_p$)
$I(t)$ is the current
$Q_n, Q_p$ are capacities of the electrodes.

$$ V(t) = U_p (x_p) - U_n (x_n) - I(t)R $$
- $V(t)$ is the cell voltage
- $U_p(x_p)$ is the OCV of the electrode
- $R$ is the resistance of the battery

In [97]:
import pybamm as pb

In [98]:
# defining dimensionless variable
x_n = pb.Variable("Negative electrode stoichiometry")
x_p = pb.Variable("Positive electrode stoichiometry")

Q_n = pb.Parameter("Negative electrode capacity [A.h]")
Q_p = pb.Parameter("Positive electrode capacity [A.h]")
R = pb.Parameter("Electrode resistance [Ohm]")
x_n_0 = pb.Parameter("Initial negative electrode stoichiometry")
x_p_0 = pb.Parameter("Initial positive electrode stoichiometry")

# function parameters (parameters that are time varying)
i = pb.FunctionParameter("Current function [A]", {"Time [s]": pb.t})
U_p = pb.FunctionParameter("Positive electrode OCV [V]", {"x_p": x_p})
U_n = pb.FunctionParameter("Negative electrode OCV [V]", {"x_n": x_n})

By construction, PyBaMM expects the equations to be written in a very specific, with time derivatives playing a central role: ODEs must be written in explicit form: $\frac{du}{dt} = f(u,t)$

This way, only $f(u, t)$ needs to be defined (RHS for right hand side) for a given variable u, as the left hand side is assumed to be $\frac{du}{dt}$


In [99]:
# creating the model
model = pb.BaseModel("ODE model")

model.rhs[x_n] = -i / Q_n
model.initial_conditions[x_n] = x_n_0
model.rhs[x_p] = -i / Q_p
model.initial_conditions[x_p] = x_p_0

model.variables["Voltage [V]"] = U_p - U_n - i*R
model.variables["Negative electrode stoichiometry"] = x_n
model.variables["Positive electrode stoichiometry"] = x_p

print(model.rhs[x_n])
print(model.rhs[x_n].children[0].children[0].children[0])

-Current function [A] / Negative electrode capacity [A.h]
time


Event = condition that can be used to stop the solver.
- Define four events that ensure that the stoichiometries $x_n$ and $x_p$ are between 0 and 1. The simulation should stop when either reach 0 or 1.


In [100]:
stop_at_t_equal_3 = pb.Event("Stop at t = 3", pb.t - 3)
model.events = [stop_at_t_equal_3]

model.events = [
    pb.Event("Min negative stoichiometry", x_n - 0),    # x_n -0 is the condition, and solver will stop when this reaches 0
    pb.Event("Max negative stoichiometry", 1 - x_n),
    pb.Event("Min positive stoichiometry", x_p - 0),
    pb.Event("Max positive stoichiometry", 1 - x_p),
]

In [101]:
import numpy as np
def graphite_LGM50_ocp_Chen2020(sto):
    u_eq = (
        1.9793 * np.exp(-39.3631 * sto)
        + 0.2482
        - 0.0909 * np.tanh(29.8538 * (sto - 0.1234))
        - 0.04478 * np.tanh(14.9159 * (sto - 0.2769))
        - 0.0205 * np.tanh(30.4444 * (sto - 0.6103))
    )

    return u_eq

def nmc_LGM50_ocp_Chen2020(sto):
    u_eq = (
        -0.8090 * sto
        + 4.4875
        - 0.0428 * np.tanh(18.5138 * (sto - 0.5542))
        - 17.7326 * np.tanh(15.7890 * (sto - 0.3117))
        + 17.5842 * np.tanh(15.9308 * (sto - 0.3120))
    )

    return u_eq

def current(t):
    return 1 + 0.5 * pb.sin(100*t)

In [102]:
# defining the parameter values
parameter_values = pb.ParameterValues({
    "Current function [A]": current,
    "Initial negative electrode stoichiometry": 0.9,
    "Initial positive electrode stoichiometry": 0.1,
    "Negative electrode capacity [A.h]": 1,
    "Positive electrode capacity [A.h]": 1,
    "Electrode resistance [Ohm]": 0.3,
    "Positive electrode OCV [V]": nmc_LGM50_ocp_Chen2020,
    "Negative electrode OCV [V]": graphite_LGM50_ocp_Chen2020,
})

# Solving the model
sim = pb.Simulation(model, parameter_values=parameter_values)
sol = sim.solve([0, 3])
sol.plot(["Voltage [V]", "Negative electrode stoichiometry", "Positive electrode stoichiometry"])

interactive(children=(FloatSlider(value=0.0, description='t', max=0.09035979586726184, step=0.0009035979586726…